<a href="https://colab.research.google.com/github/kuolunwang/Image-Processing/blob/main/hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
S# library
import numpy as np
import cv2
from matplotlib import pyplot as plt
import math
from google.colab.patches import cv2_imshow
%matplotlib inline

In [ ]:
# load picture from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loaded the image in grayscale
image = cv2.imread('/content/drive/MyDrive/fruit on tree.tif')

In [ ]:
cv2_imshow(image)

In [ ]:
def otsu(image):
  #calculate histogram
  hist=np.zeros(256,int)
  #image = cv2.cvtColor(image,)
  row,col=image.shape
  for i in range(row):
    for j in range(col):
      hist[image[i][j]]+=1
  var = np.zeros(256)
  #otsu
  min=100000
  for i in range(256):
    tot1=tot2=me1=me2=va1=va2=0
    for j in range(i+1):
      me1+=j*hist[j]
      tot1+=hist[j]
    for j in range(i+1,256):
      me2+=j*hist[j]
      tot2+=hist[j]
    if tot1==0 or tot2==0:
      continue
    else:
      me1=me1/tot1
      me2=me2/tot2
    for j in range(i+1):
     va1+=(me1-j)**2*hist[j]
    for j in range(i+1,256):
     va2+=(me2-j)**2*hist[j]
    if tot1==0 or tot2==0:
     continue
    else:   
     value=(va1+va2)/(tot1+tot2)
    varbetween = (tot1/(row*col))*(tot2/(row*col))*(me1-me2)**2
    var[i] = varbetween
    if value<min:
     min=value
     th=i
  row,col=image.shape
  dst=np.zeros((row,col),int)
  for i in range(row):
   for j in range(col):
    if image[i][j]>th:
     dst[i][j]=255
    else:
     dst[i][j]=0

  return dst, var

In [ ]:
image_r = image[:, :, 2]

In [ ]:
cv2_imshow(image_r)

In [ ]:
mask, variances = otsu(image_r)

In [ ]:
cv2_imshow(mask)

In [ ]:
plt.plot(variances)
plt.plot(np.argmax(variances), np.max(variances),
         marker="X", label="maximum")
plt.title("variance between class under different threshold")
plt.xlabel("intensity")
plt.ylabel("variance")

In [ ]:
def filter(image, mask):
    filtered = image.copy()
    filtered[np.where(mask == 0)] = [50,50,50]
    return filtered

In [ ]:
filtered = filter(image, mask)

In [ ]:
cv2_imshow(filtered)

In [ ]:
def kmeans(img, T=1):
    mask = None
    iteration = 0
    previous_centroid = None
    row , col, d = img.shape
    distance = np.zeros((row, col, 2))
    rc = np.random.randint(row, size=2)
    rr = np.random.randint(col, size=2)
    centroid = np.zeros((k, d))
    for i, (c, r) in enumerate(zip(rc, rr)):
        centroid[i] = img[c, r]
        
    while 1:
        for i in range(2):
            distance[:, :, i] = np.linalg.norm(img-centroid[i], axis=2)
        mask = np.argmin(distance, axis=2)

        for i in range(2):
            centroid[i] = np.mean(img[np.where(mask == i)], axis=0)

        if previous_centroid is not None:
            error = np.linalg.norm(previous_centroid-centroid, axis=1)
            print(iteration, error)
            if np.max(error) < T:
                break

        iteration += 1
        previous_centroid = centroid.copy()

    v = np.linalg.norm(centroid, axis=1)
    if v[0] > v[1]:
        mask = 1 - mask
    return mask*255

In [ ]:
kmeans_mask_1 = kmeans(image, T=1)
cv2_imshow(kmeans_mask_1)
kmean_filter_1 = filter(image, kmeans_mask_1)
cv2_imshow(kmean_filter_1)

In [ ]:
kmeans_mask_5 = kmeans(image, T=5)
cv2_imshow(kmeans_mask_5)
kmean_filter_5 = filter(image, kmeans_mask_5)
cv2_imshow(kmean_filter_5)

In [ ]:
kmeans_mask_10 = kmeans(image, T=10)
cv2_imshow(kmeans_mask_10)
kmean_filter_10 = filter(image, kmeans_mask_10)
cv2_imshow(kmean_filter_10)